In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import sys
sys.path.insert(0, '../src')

In [12]:
import pandas as pd
import numpy as np
import torch

import logging

from gulpio import GulpDirectory
from torchvideo.samplers import frame_idx_to_list

from config.application import FeatureConfig
from config.jsonnet import load_jsonnet
from attribution.characteristic_function_shapley_value_attributor import CharacteristicFunctionShapleyAttributor
from subset_samplers import ConstructiveRandomSampler, ExhaustiveSubsetSampler

from ipython_media import display_video

In [13]:
gulp_dir = GulpDirectory('../datasets/ssv2/gulp/validation/')

In [14]:
config_dict = load_jsonnet("../configs/feature_tsn.jsonnet")
config = FeatureConfig(**config_dict)

In [15]:
device = torch.device('cuda:0')

In [16]:
dataset = config.dataset.instantiate()
test_sampler = config.frame_samplers.test.instantiate()
val = dataset.validation_dataset(sampler=test_sampler)
model = config.get_model().to(device).eval()

In [17]:
class_priors = pd.read_csv('../datasets/ssv2/class-priors.csv', index_col='class', squeeze=True).sort_index().values
class_priors.shape

(174,)

In [18]:
frame_features, label_dict = val[8]
uid = label_dict['uid']
gt = label_dict['action']
video = np.stack(gulp_dir[uid][0])
sampled_frame_idxs = frame_idx_to_list(test_sampler.sample(len(video)))

print(frame_features.shape)
print(label_dict)

(8, 256)
{'action': 46, 'label': 46, 'uid': '100049'}


In [19]:
display_video(video, fps=14)

(29, 240, 426, 3)
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4

Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [20]:
with torch.no_grad():
    logits = model(torch.from_numpy(frame_features).to(device).unsqueeze(0))
probs = torch.softmax(logits, -1)
pred = logits.argmax().cpu().numpy().item()
pred, probs[0, pred].item()

(5, 0.4716247320175171)

In [21]:
subset_sampler = ExhaustiveSubsetSampler(device=device)
#subset_sampler = ConstructiveRandomSampler(max_samples=128, device=device)

In [23]:
def characteristic_fn(batch_subseq_features):
    # batch_subseq_features: (N, F, D)
    assert batch_subseq_features.ndim == 3
    if batch_subseq_features.shape[1] == 0:
        return torch.tensor([class_priors], device=device)
    with torch.no_grad():
        output = torch.softmax(model(batch_subseq_features), dim=-1)
    assert output.ndim == 2
    return output
    
    
attributor = CharacteristicFunctionShapleyAttributor(
    characteristic_fn=characteristic_fn,
    n_classes=config.dataset.class_count,
    device=device,
    subset_sampler=subset_sampler,
)

In [26]:
x, y = attributor.explain(torch.from_numpy(frame_features).to(device))
x[:, pred].cpu().numpy()

array([-0.05466271, -0.05059849, -0.02668883, -0.1129005 ,  0.17295125,
        0.16388573,  0.19657245,  0.17674303], dtype=float32)